In [7]:
import sys
sys.path.append('/home/shloknatarajan/TxGNN-Burkitts')

from txgnn import TxData, TxGNN, TxEval

In [12]:
from txgnn import TxData, TxGNN, TxEval
import pandas as pd
import dgl
import torch
import sys

# Check if CUDA is available
device = 'cpu'
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = 'cuda:0'
else:
    print("CUDA is not available. Using CPU.")


CUDA is not available. Using CPU.


In [13]:
%reload_ext autoreload
%autoreload 2


## Loading a Pretrained Model
<p>Saved models available at checkpoints_all_seeds/</p>

In [17]:
tx_data = TxData('./data')
tx_data.prepare_split(split='random', seed=1)
tx_gnn = TxGNN(data=tx_data, weight_bias_track=False, proj_name='TxGNN', exp_name='TxGNN', device=device)


Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Creating DGL graph....
Done!


```self.g_valid_pos, self.g_valid_neg = evaluate_graph_construct(self.df_valid, self.G, 'fix_dst', 1, self.device)``` <br>
This is the line that's failing when loading a pretrained model

In [15]:
# First check if the data is loaded correctly
print("1. Checking tx_data attributes:")
print("G:", tx_data.G)
print("\ndf_train shape:", tx_data.df_train.shape)
print("\nFirst few rows of df_train:")
print(tx_data.df_train.head())

# Then check the graph structure
print("\n2. Graph information:")
print("Node types:", tx_data.G.ntypes)
print("Edge types:", tx_data.G.etypes)
print("Number of nodes:", {ntype: tx_data.G.number_of_nodes(ntype) for ntype in tx_data.G.ntypes})
print("Number of edges:", {etype: tx_data.G.number_of_edges(etype) for etype in tx_data.G.etypes})

1. Checking tx_data attributes:
G: Graph(num_nodes={'anatomy': 14033, 'biological_process': 28642, 'cellular_component': 4176, 'disease': 17080, 'drug': 7957, 'effect/phenotype': 15311, 'exposure': 818, 'gene/protein': 27610, 'molecular_function': 11169, 'pathway': 2516},
      num_edges={('anatomy', 'anatomy_anatomy', 'anatomy'): 23328, ('anatomy', 'rev_anatomy_protein_absent', 'gene/protein'): 16531, ('anatomy', 'rev_anatomy_protein_present', 'gene/protein'): 1262006, ('biological_process', 'bioprocess_bioprocess', 'biological_process'): 87924, ('biological_process', 'rev_bioprocess_protein', 'gene/protein'): 120369, ('biological_process', 'rev_exposure_bioprocess', 'exposure'): 1351, ('cellular_component', 'cellcomp_cellcomp', 'cellular_component'): 8056, ('cellular_component', 'rev_cellcomp_protein', 'gene/protein'): 69328, ('cellular_component', 'rev_exposure_cellcomp', 'exposure'): 9, ('disease', 'disease_disease', 'disease'): 53522, ('disease', 'disease_phenotype_negative', 'eff

In [16]:
mappings = tx_data.retrieve_id_mapping()
id2name_drug = mappings['id2name_drug']
idx2id_drug = mappings['idx2id_drug']

In [9]:
print(idx2id_drug)

{5810.0: 'DB09130', 5819.0: 'DB09140', 168.0: 'DB00180', 228.0: 'DB00240', 241.0: 'DB00253', 312.0: 'DB00324', 380.0: 'DB00394', 429.0: 'DB00443', 572.0: 'DB00588', 575.0: 'DB00591', 580.0: 'DB00596', 604.0: 'DB00620', 619.0: 'DB00635', 632.0: 'DB00648', 647.0: 'DB00663', 670.0: 'DB00687', 723.0: 'DB00741', 827.0: 'DB00846', 841.0: 'DB00860', 877.0: 'DB00896', 994.0: 'DB01013', 1026.0: 'DB01047', 1198.0: 'DB01222', 1324.0: 'DB01384', 1343.0: 'DB01410', 3609.0: 'DB05688', 5625.0: 'DB08906', 5805.0: 'DB09124', 7481.0: 'DB13867', 7695.0: 'DB14538', 7696.0: 'DB14539', 7697.0: 'DB14540', 7698.0: 'DB14541', 7699.0: 'DB14542', 7700.0: 'DB14543', 7701.0: 'DB14544', 7725.0: 'DB14631', 7743.0: 'DB14669', 7934.0: 'DB15566', 6023.0: 'DB09479', 151.0: 'DB00162', 4076.0: 'DB06755', 152.0: 'DB00163', 2682.0: 'DB03917', 3336.0: 'DB05076', 4257.0: 'DB06985', 243.0: 'DB00255', 267.0: 'DB00279', 437.0: 'DB00451', 570.0: 'DB00586', 842.0: 'DB00861', 1071.0: 'DB01093', 1445.0: 'DB01583', 1495.0: 'DB01645',

In [18]:
# Load pretrained weights
tx_gnn.load_pretrained('./checkpoints_all_seeds/TxGNN_1_random')

In [ ]:
# Create evaluator and get predictions
tx_eval = TxEval(model=tx_gnn)
result = tx_eval.eval_disease_centric(
    disease_idxs=[7243.0],  # Burkitt's lymphoma
    relation='indication',
    save_result=False,
    verbose=True
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
result

,ID,Name,Ranked List,AUROC,AUPRC,Accuracy,Sensitivity,Specificity,F1,PPV,...,AP@50,Hits@50,Missed@50,Recall@100,Recall_Random@100,Enrichment@100,MRR@100,AP@100,Hits@100,Missed@100
20181.0,20181.0,mesenchymatous palpebral tumor,"[Aminolevulinic acid, Moxetumomab Pasudotox, M...",-1,-1,1.0,NaN,1.0,0.0,NaN,...,-1,[],[],-1,-1,-1,-1,-1,[],[]
